## User tutorial 1 : simple two components model

This tutorial show how to set-up and solve a very simple model using EESREP.

This model takes two components:

-   source : provides the requested energy;
-   load : requests a pre-defined amount of energy.

The structure of the model can be illustrated as follow:

    source --> load

####   Imports

In [1]:
import math

import pandas as pd

from eesrep import Eesrep

####   Create model

A EESREP object is created providing the interface to use. This interface will be used to select which python module is used to set-up the MILP model.

As not provided, the optimisation function here will be minimised.

The Eesrep object will store the structure of the system (components and links), and will convert it in a MILP model to compute its optimal state.

In [2]:
model = Eesrep(interface="docplex")

To set up the model, we need to create its components.

A component is created using its constructor. The built-in components can be found in eesrep.components.*

In this tutorial we will use the following components:

-   Source : provides the energy requested by the system
-   FatalSink : Requests a given amount of energy defined in a time serie

After creation, the component can be provided to the Eesrep model.

The options names are writen in this file for better readability, but are not necessary.

In [3]:
from eesrep.components.sink_source import FatalSink, Source

source = Source(name = "source", p_min=0., p_max=100., price=0.)
model.add_component(source)

When a time serie is provided, it is defined using a pandas dataframe containing two columns: **time** and **value**. The time column does not need to be the same for each component, an interpolation of these data will be done before solving.

In [4]:
fs_df = pd.DataFrame({"time": list(range(1001)), "value": [5 + 2* math.sin(i/30) for i in range(1001)]})

load = FatalSink(name = "load",
                    sink_flow=fs_df)

model.add_component(load)

The created components now need to see their input/output linked to each others. It will create an equality constraint between them.

A multiplication coefficient/offset can be added using the two last parameters. Here we would have:

    power_out * 1. + 0. = power_in

In [5]:
model.add_link(io_1=source.power_out, 
                io_2=load.power_in, 
                factor=1., 
                offset=0.)

####   Set up simulation: rolling horizons

An EESREP system optimisation on a global time horizon can be separated in several rolling horizons with a constant time delay between two, with or without overlapping between two consecutive horizons.

The rolling horizon is defined by four parameters:

-   time_step (float) : length of a time step, make sure its value is consistent with the one provided in the time series.
-   time_shift (int) : time steps skipped between two horisons solved.
-   future_size (int) : number of time steps solved in each MILP resolution.
-   horizon_count (int) : number of rolling horizon computed.

The following illustrates a resolution using 3 rolling horizons (horizon_count = 3) of 9 steps each (future_size = 9), and a time shift of two steps between two rolling horizons (time_shift = 2).

        |---|---|---|---|---|---|---|---|---| Rolling horizon 1
                |---|---|---|---|---|---|---|---|---| Rolling horizon 2
                        |---|---|---|---|---|---|---|---|---| Rolling horizon 3

In this example, he two first steps of the first rolling horizon will be used as history for the second horizon, and the two first steps of both two first rolling horizons will be used as history for rolling horizon 3.

Before solving your first case, it is required to define its time range parameters.

In [6]:
model.define_time_range(time_step = 1., 
                        time_shift = 1, 
                        future_size = 1000, 
                        horizon_count = 1)

####   Solve and get results

The model solve function will solve each rolling horizon requested in the *define_time_range*.

In [7]:
model.solve()

Running first time step


The results of the solved rolling horison can be accessed through the *get_results* function. Two formats are available:

-   **Pandas dataframe** : every component input/output are provided in a single dataframe, the name of the componnent and the I/O name are separated with an underscore;
-   **Dictionnary** : a dictionnary, with every component name as keys, containing a dictionnary of every input/output of the given component. 

In [8]:
results = model.get_results(as_dataframe=True)
print(results)

     source_power_out  load_power_in    time
0            5.066654       5.066654     1.0
1            5.133235       5.133235     2.0
2            5.199667       5.199667     3.0
3            5.265877       5.265877     4.0
4            5.331792       5.331792     5.0
..                ...            ...     ...
995          6.954685       6.954685   996.0
996          6.939491       6.939491   997.0
997          6.922142       6.922142   998.0
998          6.902657       6.902657   999.0
999          6.881059       6.881059  1000.0

[1000 rows x 3 columns]
